In [42]:
from config import load_dotenv

In [43]:
data = r"C:\Users\CHINMAYI SURYAMATH\Desktop\ML PROJECT ECOMMERCE\NN_pdf\Data\table1.pdf"

In [44]:
print(data[0].page_content)


AttributeError: 'str' object has no attribute 'page_content'

In [45]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(data)
docs = loader.load()

print(len(docs))


1


In [46]:
print(docs[0].page_content)


COMMERCIAL LINES POLICY- COMMON POLICY DECLARATIONS 
NAUTILUS INSURANCE COMPANY 
Scottsdale, Arizona 
Policy No. NN893805 Transaction Type: Renewal 
Renewal of Policy# NN89 3427 
Rewrite of Policy# ______ _ 
Inspection Ordered: 
l:RlYes 0No This insurance contract is issued pursuant to the 
Maine Insurance Laws by an insurer neither 
licensed by norunder the jurisdiction of the 
Maine Bureau of Insurance. 
Cross Ref. Policy# ______ _ 
NIC Quote# 
Named Insured and Mailing Address 
(No., Street, Town or City, County, State, Zip Code) 
Estate of John S. Norton, Sr. 
Beth Preston 
Agent and Mailing Address 
Agency No. 1803-00 
(No., Street, Town or City, County, State, Zip Code) 
Breckenridge Insurance Services, LLC 
PO Box 9010 
1 Acadia Commons 
Westbrook 
Policy 
ME 04098 
NO FLAT CANCELLATION 
Period: From 08/08/2019 to o 8 / o 8 / 2 o 2 o at 12:01 A.M. Standard Time at your mailing address shown above. 
Business Description: Property Owner 
Form of Business: Individual 
Tax Statel§_ 

TEXT SPLITTER

In [47]:
parameters = {
    "chunk_size": 250,
    "chunk_overlap": 50,
    "separators":["\n\n", "\n", " ", ""],    # Split priority
   "keep_separator":"end",                  # Keep separators at end
    "add_start_index":True

}

In [48]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(**parameters)
chunks = text_splitter.split_documents(docs)
print(chunks[0].page_content)



COMMERCIAL LINES POLICY- COMMON POLICY DECLARATIONS 
NAUTILUS INSURANCE COMPANY 
Scottsdale, Arizona 
Policy No. NN893805 Transaction Type: Renewal 
Renewal of Policy# NN89 3427 
Rewrite of Policy# ______ _ 
Inspection Ordered:


EMBEDDING

In [49]:
from langchain_openai import OpenAIEmbeddings
from config import OPENAI_API_KEY

embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY,model="text-embedding-3-small")

VECTORSTORE

In [50]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embeddings)

RETRIVER

In [51]:
retriever1 = vectorstore.as_retriever(search_type="similarity",search_kwargs={"k": 3})

In [52]:
retriever2 = vectorstore.as_retriever(search_type="mmr",search_kwargs={"k": 5})

In [58]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

def pipeline(prompt):
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=OPENAI_API_KEY)

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever2 ,
        chain_type="stuff"  # simplest way: stuff retrieved docs into prompt
    )

    query01 = prompt
    #print("Q:", query01)
    print("A:", qa_chain.run(query01),"\n\n")

    



In [61]:
docs = retriever2.get_relevant_documents("Policy number")
for i, d in enumerate(docs):
    print(f"Doc {i}:\n", d.page_content[:500], "\n---\n")


C:\Users\CHINMAYI SURYAMATH\AppData\Local\Temp\ipykernel_10840\114819149.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever2.get_relevant_documents("Policy number")


Doc 0:
 Maine Bureau of Insurance. 
Cross Ref. Policy# ______ _ 
NIC Quote# 
Named Insured and Mailing Address 
(No., Street, Town or City, County, State, Zip Code) 
Estate of John S. Norton, Sr. 
Beth Preston 
Agent and Mailing Address 
Agency No. 1803-00 
---

Doc 1:
 FORM(S) AND FORMS AND ENDORSEMENTS, IF ANY, ISSUED TO FORM A PART THEREOF, COMPLETE THE ABOVE NUMBERED POLICY. 
Includes copyrighted material of Insurance Services Office, Inc., with its permission. 
EXHIBIT 
E001 (02/14) 
---

Doc 2:
 Policy 
ME 04098 
NO FLAT CANCELLATION 
Period: From 08/08/2019 to o 8 / o 8 / 2 o 2 o at 12:01 A.M. Standard Time at your mailing address shown above. 
Business Description: Property Owner 
Form of Business: Individual 
Tax Statel§_ 
---

Doc 3:
 $ _______ _ 
$ _______ _ 
$ _______ _ 
$ _______ _ 
$ _______ _ 
Tax & Fee Schedule TOTAL ADVANCE PREMIUM $ 1 100. 00 
Policy Fee 
Inspection Fee Surplus Lines Tax 
$ 65. oo Minimum & Deposit 150.00 
39 · 45 TOTAL TAXES & FEES $ _____ 2_5_4_._4_

In [56]:
pipeline("""You are an expert pdf reader and text and table data extractor . 
         Extract  policy number from the attached document.
          Policy No. NN893806 """)

Q: You are an expert pdf reader and text and table data extractor . 
         Extract  policy number from the attached document.
          Policy No. NN893806 
A: The policy number extracted from the document is NN893805. 




In [60]:
pipeline("""
You are an expert PDF reader and structured data extractor.

Task:
- Read the provided PDF text.
- Extract the following fields:
  - policy_number
  - policy_holder_name
  - start_date
  - end_date
  - premium_amount
  - tables (if present)
-if you find other important data values include in extraction process
         

"""

)

A: I don't know. 




In [65]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

def pipeline(prompt):
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0,
        api_key=OPENAI_API_KEY
    )

    # Custom structured prompt
    template = """
    You are an expert PDF reader and structured data extractor.

    Task:
    - Read the provided PDF text.
    - Extract the following fields:
      - policy_number
      - policy_holder_name
      - start_date
      - end_date
      - premium_amount
      - tables (if present)
    - If you find other important values (like business type, tax, fees, agent name, etc.) include them too.

    Output:
    Return ONLY valid JSON. If a field is not found, use null.

    Example format:
    {{
      "policy_number": "",
      "policy_holder_name": "",
      "start_date": "",
      "end_date": "",
      "premium_amount": "",
      "tables": [],
      "other_fields": {{}}
    }}

    --------------------
    Context:
    {context}
    --------------------
    Question: {question}
    """

    QA_PROMPT = PromptTemplate(
        input_variables=["context", "question"],
        template=template
    )

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever2,
        chain_type="stuff",
        chain_type_kwargs={"prompt": QA_PROMPT},
        return_source_documents=True
    )

    result = qa_chain(prompt)

    print("A:", result["result"], "\n\n")
    print("Source Docs:\n")
    for i, doc in enumerate(result["source_documents"]):
        print(f"Doc {i}:\n", doc.page_content[:], "\n---\n")

# Run pipeline
pipeline("Extract structured policy details from the document")


A: ```json
{
  "policy_number": "NN893805",
  "policy_holder_name": null,
  "start_date": null,
  "end_date": null,
  "premium_amount": "1100.00",
  "tables": [],
  "other_fields": {
    "transaction_type": "Renewal",
    "previous_policy_number": "NN893427",
    "policy_fee": "65.00",
    "minimum_deposit": "150.00",
    "surplus_lines_tax": null,
    "total_taxes_and_fees": "254.45",
    "insurer": "Nautilus Insurance Company",
    "location": "Scottsdale, Arizona"
  }
}
``` 


Source Docs:

Doc 0:
 FORM(S) AND FORMS AND ENDORSEMENTS, IF ANY, ISSUED TO FORM A PART THEREOF, COMPLETE THE ABOVE NUMBERED POLICY. 
Includes copyrighted material of Insurance Services Office, Inc., with its permission. 
EXHIBIT 
E001 (02/14) 
---

Doc 1:
 Rewrite of Policy# ______ _ 
Inspection Ordered: 
l:RlYes 0No This insurance contract is issued pursuant to the 
Maine Insurance Laws by an insurer neither 
licensed by norunder the jurisdiction of the 
Maine Bureau of Insurance. 
---

Doc 2:
 parties shall

In [66]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

def pipeline(prompt):
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0,
        api_key=OPENAI_API_KEY
    )

    # Custom structured prompt
    template = """
    You are an expert PDF reader and structured data extractor.

    Task:
    - Read the provided PDF text.
    - Extract the following fields:
      - policy_number
      - policy_holder_name
      - start_date
      - end_date
      - premium_amount
      - tables (if present)
    - If you find other important values (like business type, tax, fees, agent name, etc.) include them too.

    Output:
    Return in JSON format. If a field is not found, use null.

    Example format:
    {{
      "policy_number": "",
      "policy_holder_name": "",
      "start_date": "",
      "end_date": "",
      "premium_amount": "",
      "tables": [],
      "other_fields": {{}}
    }}

    --------------------
    Context:
    {context}
    --------------------
    Question: {question}
    """

    QA_PROMPT = PromptTemplate(
        input_variables=["context", "question"],
        template=template
    )

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever2,
        chain_type="stuff",
        chain_type_kwargs={"prompt": QA_PROMPT},
        return_source_documents=True
    )

    result = qa_chain(prompt)

    print("A:", result["result"], "\n\n")
    

# Run pipeline
pipeline("Extract structured policy details from the document")


A: ```json
{
  "policy_number": "NN893805",
  "policy_holder_name": null,
  "start_date": null,
  "end_date": null,
  "premium_amount": "1100.00",
  "tables": [],
  "other_fields": {
    "transaction_type": "Renewal",
    "previous_policy_number": "NN893427",
    "policy_fee": "65.00",
    "minimum_deposit": "150.00",
    "surplus_lines_tax": null,
    "total_taxes_and_fees": "254.45",
    "insurer": "Nautilus Insurance Company",
    "location": "Scottsdale, Arizona"
  }
}
``` 




In [69]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

def pipeline(prompt):
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0,
        api_key=OPENAI_API_KEY
    )

    # Generalized extraction prompt
    template = """
    You are an expert PDF reader and information extractor.

    Task:
    - Carefully read the provided PDF content.
    - Extract and summarize all important information, including:
      • Key fields (like policy numbers, dates, names, addresses, fees, amounts, etc.)
      • Contextual details
      • Tabular data (represent tables clearly in text or structured form)
      • Any other significant information relevant to the document.

    Present the extracted information in a clear, structured, and human-readable format.
    
    --------------------
    Context:
    {context}
    --------------------
    Question: {question}
    """

    QA_PROMPT = PromptTemplate(
        input_variables=["context", "question"],
        template=template
    )

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever2,
        chain_type="stuff",
        chain_type_kwargs={"prompt": QA_PROMPT},
        return_source_documents=True
    )

    result = qa_chain(prompt)

    print("A:", result["result"], "\n\n")
    print("Source docs used:\n")
    for i, doc in enumerate(result["source_documents"]):
        print(f"Doc {i}:\n", doc.page_content[:300], "\n---\n")


# Run pipeline
pipeline("Extract all important information from the document")


A: Here is the extracted and summarized information from the provided PDF content:

### Document Summary

**Title:** EXHIBIT E001 (02/14)

**Disclaimer:** 
- The information is derived from sources believed to be correct but is not guaranteed.
- Interested parties should rely on their own information and judgment.
- Property is sold on an "AS IS, WHERE IS" basis.

### Key Fields

- **Policy Number:** (Not provided, indicated as "______")
- **Inspection Ordered:** Yes (indicated as "l:RlYes") / No (indicated as "0No")
- **Insurance Contract Issuer:** Not licensed by or under the jurisdiction of the Maine Bureau of Insurance.
  
### Financial Information

- **Total Advance Premium:** $1,100.00
- **Policy Fee:** $65.00
- **Inspection Fee:** (Not specified)
- **Surplus Lines Tax:** (Not specified)
- **Minimum & Deposit:** $150.00
- **Total Taxes & Fees:** $254.45 (indicated as "$ _____ 2_5_4_._4_5")

### Additional Notes

- The document includes forms and endorsements that complete the abo